In [1]:
import sys
from datetime import datetime

import sapien.core as sapien
from sapien.core import Pose
from Tools import misc, ForwardKinematics, ModelSim, NumForwardDynamicsDer, MathForwardDynamics, DerivativeFactory, \
    DerivativeWorker
import numpy as onp
import jax.numpy as np
from jax import jit, jacfwd, jacrev, grad
from ilq import ILQG_Human
import timeit
import os
import matplotlib.pyplot as plt

os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "0.4"
os.environ["XLA_FLAGS"] = ("--xla_cpu_multi_thread_eigen=false "
                           "intra_op_parallelism_threads=12")

sim = sapien.Engine()
renderer = sapien.OptifuserRenderer()
sim.set_renderer(renderer)
render_controller = sapien.OptifuserController(renderer)

Using default glsl path /home/zack/anaconda3/envs/ml/lib/python3.7/site-packages/sapien/glsl_shader/130


In [2]:
DEBUG = False


def create_scene(timestep, visual):
    s = sim.create_scene()
    s.add_ground(-1)
    s.set_timestep(timestep)

    loader = s.create_urdf_loader()
    loader.fix_root_link = False
    if visual:
        loader.collision_is_visual = True
        s.set_ambient_light([0.5, 0.5, 0.5])
        s.set_shadow_light([0, 1, -1], [0.5, 0.5, 0.5])

    # build
    robot = loader.load("../assets/Humanoid/humanoid.urdf")
    robot.set_pose(Pose([0, 0, -0.7], [0, 0.7071068, 0, 0.7071068]))
    robot.set_qpos(onp.array([-0.07463598, 0.71799153, 0.00224937, 0.24503642, -0.69970787, -0.07047031,
                              0.02011428, -0.2995666,  -0.07851388,  0.37046608,  1.0063629,   0.7287718,
                              -0.8464428,  -0.6261912,  -0.61831105,  1.5032201,   1.2024844,   1.2001486,
                              -1.5865482,   0.00861856, -0.13306628]))

    for joint in robot.get_joints():
        joint.set_drive_property(stiffness=0, damping=10)

    for _ in range(int(1 / timestep) * 4):
        s.step()

    return s, robot


# renderer_timestep = 1 / 60
optim_timestep = 1 / 100
sim_timestep = 1 / 30
render_steps = int(sim_timestep / optim_timestep) + 1
s0, robot = create_scene(optim_timestep, True)

render_controller.set_camera_position(0, -5, 0)
render_controller.set_current_scene(s0)

state = misc.get_state(robot)
num_x = len(state)
num_u = robot.dof

In [3]:
render_controller.show_window()
render_controller.focus(robot.get_links()[0])
while not render_controller.should_quit:
    s0.step()
    s0.update_render()
    render_controller.render()

print(robot.get_qpos())

[ 0.48666376  0.79409945  0.02513883  0.70838255 -0.5492408   0.89989704
 -3.0685534   0.09497914  0.24372202  0.56275046 -6.1733613  -0.07287689
 -0.21311814 -2.1692207   2.8448288   3.9054487  -4.7923527  -0.89456135
  1.1735348   0.70226294  0.16475289]
